# Week 6 - Classification models  

## Part 3: Travel mode choice - Probit regression

In [ ]:
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/GZWqSVl9GPU" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

In this part we will revisit our real world problem of travel model choice. 

The first part is very similar to previous notebook for part 2: loading data, preprocessing, train/test split, etc. However, in this part, we will consider a Probit regression model. For the sake of simplicty, lets assume that we are just interested in distinguishing between car vs non-car (binary classification problem).

Lets just start running the parts corresponding to imports, data loading, preprocessing, train/test split, etc.

Import required libraries:

In [ ]:
# Install Pyro, if necessary
!pip install pyro-ppl

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import linear_model
import seaborn as sns
import torch

import pyro
import pyro.distributions as dist
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
from pyro.infer import MCMC, NUTS, HMC, SVI, Trace_ELBO
from pyro.optim import Adam, ClippedAdam

# fix random generator seed (for reproducibility of results)
np.random.seed(42)

# matplotlib style options
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 10)

Load data:

In [ ]:
# load csv
df = pd.read_csv("http://mlsm.man.dtu.dk/mbml/modechoice_data.csv")
df.head()

,Unnamed: 0,individual,hinc,psize,ttme_air,invc_air,invt_air,gc_air,ttme_train,invc_train,invt_train,gc_train,ttme_bus,invc_bus,invt_bus,gc_bus,invc_car,invt_car,gc_car,mode_chosen
0,0,70.0,30.0,4.0,10.0,61.0,80.0,73.0,44.0,24.0,350.0,77.0,53.0,19.0,395.0,79.0,4.0,314.0,52.0,1.0
1,1,8.0,15.0,4.0,64.0,48.0,154.0,71.0,55.0,25.0,360.0,80.0,53.0,14.0,462.0,84.0,4.0,351.0,57.0,2.0
2,2,62.0,35.0,2.0,64.0,58.0,74.0,69.0,30.0,21.0,295.0,66.0,53.0,24.0,389.0,83.0,7.0,315.0,55.0,2.0
3,3,61.0,40.0,3.0,45.0,75.0,75.0,96.0,44.0,33.0,418.0,96.0,53.0,28.0,463.0,98.0,5.0,291.0,49.0,1.0
4,4,27.0,70.0,1.0,20.0,106.0,190.0,127.0,34.0,72.0,659.0,143.0,35.0,33.0,653.0,104.0,44.0,592.0,108.0,1.0


Preprocess data:

In [ ]:
# separate between features/inputs (X) and target/output variables (y)
mat = df.values
X = mat[:,2:-1]
print(X.shape)
y = mat[:,-1].astype("int")
print(y.shape)
ind = mat[:,1].astype("int")
print(ind.shape)

(394, 17)
(394,)
(394,)


### This part is important!

This is where we turn our previous 4-class problem into a binary classification problem: car vs non-car

In [ ]:
# transform to binary problem: car vs non-car
y = (y == 4).astype("int")

In [ ]:
# standardize input features
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X = (X - X_mean) / X_std

Train/test split:

In [ ]:
train_perc = 0.66 # percentage of training data
split_point = int(train_perc*len(y))
perm = np.random.permutation(len(y))
ix_train = perm[:split_point]
ix_test = perm[split_point:]
X_train = X[ix_train,:]
X_test = X[ix_test,:]
y_train = y[ix_train]
y_test = y[ix_test]
print("num train: %d" % len(y_train))
print("num test: %d" % len(y_test))

num train: 260
num test: 134


Again, for the purpose of comparison, we run the logistic regression method from sklearn. But note that although sklearn has an implementation of logistic regression, it is not a Bayesian approach, nor does it support probit regression or some other variant that you may think is more appropriate for your particular problem. On the other hand, Pyro offers us complete flexibility!

In [ ]:
# create and fit logistic regression model
logreg = linear_model.LogisticRegression(solver='lbfgs', multi_class='auto')
logreg.fit(X_train, y_train)

# make predictions for test set
y_hat = logreg.predict(X_test)
print("predictions:", y_hat)
print("true values:", y_test)

# evaluate prediction accuracy
print("Accuracy:", 1.0*np.sum(y_hat == y_test) / len(y_test))

predictions: [0 0 0 1 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0
 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 1 1 1
 1 1 0 0 0 0 1 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 1 0 1 0 1
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
true values: [1 0 0 1 1 0 0 0 1 1 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 1 1 1 1 0 0 0
 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 1 0 1
 1 1 0 0 1 0 1 1 0 1 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 1 1 1 0 1
 1 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 1]
Accuracy: 0.7164179104477612


Ok, time to implement binary logistic regression in Pyro!

Your turn now :-)

Note: don't forget to include an explicit intercept parameter $\alpha$ in the model!

In [ ]:
def model(X, obs=None):
    // TODO

Let's prepare the data for Pyro by converting it into PyTorch tensors:

In [ ]:
# Prepare data for Pyro
X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).float()

Run approximate Bayesian inference using SVI:

In [ ]:
# Define guide function
guide = AutoMultivariateNormal(model)

# Reset parameter values
pyro.clear_param_store()

# Define the number of optimization steps
n_steps = 30000

# Setup the optimizer
adam_params = {"lr": 0.001}
optimizer = ClippedAdam(adam_params)

# Setup the inference algorithm
elbo = Trace_ELBO(num_particles=1)
svi = SVI(model, guide, optimizer, loss=elbo)

# Do gradient steps
for step in range(n_steps):
    elbo = svi.step(X_train, y_train)
    if step % 1000 == 0:
        print("[%d] ELBO: %.1f" % (step, elbo))

[0] ELBO: 601.9
[1000] ELBO: 304.9
[2000] ELBO: 214.1
[3000] ELBO: 204.0
[4000] ELBO: 196.2
[5000] ELBO: 196.3
[6000] ELBO: 194.9
[7000] ELBO: 193.8
[8000] ELBO: 194.2
[9000] ELBO: 192.0
[10000] ELBO: 186.3
[11000] ELBO: 189.9
[12000] ELBO: 190.2
[13000] ELBO: 190.1
[14000] ELBO: 191.8
[15000] ELBO: 189.9
[16000] ELBO: 190.7
[17000] ELBO: 187.5
[18000] ELBO: 190.6
[19000] ELBO: 187.4
[20000] ELBO: 188.1
[21000] ELBO: 189.7
[22000] ELBO: 188.6
[23000] ELBO: 185.0
[24000] ELBO: 190.0
[25000] ELBO: 189.0
[26000] ELBO: 188.2
[27000] ELBO: 185.6
[28000] ELBO: 188.5
[29000] ELBO: 189.3


Upon convergence, we can use the ```Predictive``` class to extract samples from posterior:

In [ ]:
from pyro.infer import Predictive

predictive = Predictive(model, guide=guide, num_samples=2000,
                        return_sites=("alpha", "beta"))
samples = predictive(X_train, y_train)

We can now use the inferred posteriors to make predictions for the testset and compute the corresponding accuracy:

In [ ]:
alpha_hat = samples["alpha"].mean(axis=0).detach().numpy()
beta_hat = samples["beta"][:,0].mean(axis=0).detach().numpy()

In [ ]:
# make predictions for test set
y_hat = alpha_hat + np.dot(X_test, beta_hat)
y_hat = (y_hat > 0).astype(np.int)
print("predictions:", y_hat)
print("true values:", y_test)

# evaluate prediction accuracy
print("Accuracy:", 1.0*np.sum(y_hat == y_test) / len(y_test))

predictions: [0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 1 1 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 0 0 0 1 1 1
 1 1 0 0 0 0 1 1 0 1 1 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 1 0 1 0 1
 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1]
true values: [1 0 0 1 1 0 0 0 1 1 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 1 1 1 1 0 0 0
 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 1 0 1
 1 1 0 0 1 0 1 1 0 1 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 1 1 1 0 1
 1 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 1]
Accuracy: 0.7313432835820896


Nice, it seems that we are already doing better than sklearn!

Ok, now lets try a **probit regression model in Pyro**.

Can you implement it?

Hint: you need to find a way to compute the CDF of a standard Gaussian distribution (i.e. N(0,1)) in PyTorch...

In [ ]:
std_normal = torch.distributions.Normal(0,1)

def model(X, obs=None):
    // TODO

Run approximate Bayesian inference using SVI:

In [ ]:
# Define guide function
guide = AutoMultivariateNormal(model)

# Reset parameter values
pyro.clear_param_store()

# Define the number of optimization steps
n_steps = 30000

# Setup the optimizer
adam_params = {"lr": 0.0005}
optimizer = ClippedAdam(adam_params)

# Setup the inference algorithm
elbo = Trace_ELBO(num_particles=1)
svi = SVI(model, guide, optimizer, loss=elbo)

# Do gradient steps
for step in range(n_steps):
    elbo = svi.step(X_train, y_train)
    if step % 1000 == 0:
        print("[%d] ELBO: %.1f" % (step, elbo))

[0] ELBO: 1373.6
[1000] ELBO: 619.0
[2000] ELBO: 415.9
[3000] ELBO: 279.5
[4000] ELBO: 240.3
[5000] ELBO: 213.6
[6000] ELBO: 207.9
[7000] ELBO: 210.5
[8000] ELBO: 205.8
[9000] ELBO: 204.6
[10000] ELBO: 203.7
[11000] ELBO: 200.1
[12000] ELBO: 203.0
[13000] ELBO: 201.0
[14000] ELBO: 199.7
[15000] ELBO: 200.1
[16000] ELBO: 197.2
[17000] ELBO: 200.3
[18000] ELBO: 196.0
[19000] ELBO: 196.3
[20000] ELBO: 194.3
[21000] ELBO: 197.9
[22000] ELBO: 197.6
[23000] ELBO: 197.5
[24000] ELBO: 196.1
[25000] ELBO: 196.9
[26000] ELBO: 197.5
[27000] ELBO: 197.2
[28000] ELBO: 195.7
[29000] ELBO: 197.0


Upon convergence, we can use the ```Predictive``` class to extract samples from posterior:

In [ ]:
from pyro.infer import Predictive

predictive = Predictive(model, guide=guide, num_samples=2000,
                        return_sites=("alpha", "beta"))
samples = predictive(X_train, y_train)

We can now use the inferred posteriors to make predictions for the testset and compute the corresponding accuracy:

In [ ]:
alpha_hat = samples["alpha"].mean(axis=0).detach().numpy()
beta_hat = samples["beta"][:,0].mean(axis=0).detach().numpy()

In [ ]:
# make predictions for test set
y_hat = alpha_hat + np.dot(X_test, beta_hat)
y_hat = (y_hat > 0).astype(np.int)
print("predictions:", y_hat)
print("true values:", y_test)

# evaluate prediction accuracy
print("Accuracy:", 1.0*np.sum(y_hat == y_test) / len(y_test))

predictions: [0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 1 1 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 0 0 0 1 1 1
 1 1 0 0 0 0 1 1 0 1 1 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 1 0 1 0 1
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
true values: [1 0 0 1 1 0 0 0 1 1 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 1 1 1 1 0 0 0
 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 1 0 1
 1 1 0 0 1 0 1 1 0 1 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 1 1 1 0 1
 1 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 1]
Accuracy: 0.7238805970149254


How are your results in comparison to the version with the logistic sigmoid?

In some cases, using a probit function instead of the logistic sigmoid can make a significant difference. In other cases, it doesn't... You have to consider what makes more sense to the specific problem that you are trying to solve. Or, we can just try different approaches! That is just fine... Pyro makes it very easy to try all these different variants.